In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Import & Data preprocessing

In [2]:
df1 = pd.read_csv('permtempdata_logit_stepwise.csv', engine='python')
df2 = pd.read_csv('bigsmalldata_logit_stepwise.csv', engine='python')

In [3]:
df1 = df1.drop('Unnamed: 0',axis=1)
df2 = df2.drop('Unnamed: 0',axis=1)

In [4]:
x1 = pd.get_dummies(df1['uniMajor'], prefix='uniMajor')
x2 = pd.get_dummies(df1['youthLoc'], prefix='youthLoc')
x3 = pd.get_dummies(df1['newedu'], prefix='newedu')

In [5]:
X = df1.drop(['uniMajor','youthLoc','newedu','permIdc'],axis=1)
X = X.replace(1,0)
X = X.replace(2,1)
X1 = pd.concat([X,x1,x2,x3],axis=1)

In [37]:
#더미변수로 만들어주기

x1 = pd.get_dummies(df2['uniType'], prefix='uniType')
x2 = pd.get_dummies(df2['uniMajor'], prefix='uniMajor')
x3 =  pd.get_dummies(df2['youthHouse'], prefix='youthHouse')

X = df2.drop(['uniMajor','uniType','corpSize','youthHouse'],axis=1)
X = X.replace(1,0)
X = X.replace(2,1)
X2 = pd.concat([X,x1,x2,x3],axis=1)

In [7]:
X=X1
y = df1['permIdc']

# Logistic Regression - 정규직/비정규직

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_validTest, y_train, y_validTest = train_test_split(X, y, train_size=0.6, test_size = 0.4, random_state=1234) 
X_valid, X_test, y_valid, y_test = train_test_split(X_validTest, y_validTest, train_size = 0.5, test_size = 0.5, random_state = 1234)

In [9]:
from sklearn.linear_model import LogisticRegression
m1 = LogisticRegression(max_iter = 4000)
m1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=4000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=m1, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_valid, y_valid)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.851638 using {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.821230 (0.036913) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.821230 (0.036913) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.821230 (0.036913) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.824278 (0.030234) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.824278 (0.030234) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.824278 (0.031352) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.843270 (0.021513) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.843270 (0.021513) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.842512 (0.021978) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.851638 (0.010785) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.851638 (0.010785) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.851638 (0.010785) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.851638 (0.010785) wit

In [11]:
model = LogisticRegression( C= 0.1, penalty= 'l2',solver= 'newton-cg')
model.fit(X_train, y_train)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
X.columns

Index(['gpaIdc', 'certIdc', 'abrIdc', 'internIdc', 'gradIdc', 'clubIdc',
       'uniMajor_1', 'uniMajor_2', 'uniMajor_3', 'uniMajor_4', 'uniMajor_5',
       'uniMajor_6', 'uniMajor_7', 'uniMajor_8', 'uniMajor_9', 'uniMajor_10',
       'uniMajor_19990920', 'youthLoc_1', 'youthLoc_2', 'youthLoc_3',
       'youthLoc_4', 'youthLoc_5', 'youthLoc_6', 'youthLoc_7', 'youthLoc_8',
       'youthLoc_9', 'youthLoc_10', 'youthLoc_11', 'youthLoc_12',
       'youthLoc_13', 'youthLoc_14', 'youthLoc_15', 'youthLoc_16', 'newedu_1',
       'newedu_2', 'newedu_3', 'newedu_4', 'newedu_5'],
      dtype='object')

In [13]:
standard = [1,1,1,1,1,1]+[1]+[0]*10+[1]+[0]*15+[1]+[0]*4
gpa = [0,1,1,1,1,1]+[1]+[0]*10+[1]+[0]*15+[1]+[0]*4
grad = [1,0,1,1,1,1]+[1]+[0]*10+[1]+[0]*15+[1]+[0]*4
inter = [1,1,0,1,1,1]+[1]+[0]*10+[1]+[0]*15+[1]+[0]*4
abr = [1,1,1,0,1,1]+[1]+[0]*10+[1]+[0]*15+[1]+[0]*4
cert = [1,1,1,1,0,1]+[1]+[0]*10+[1]+[0]*15+[1]+[0]*4
club = [1,1,1,1,1,0]+[1]+[0]*10+[1]+[0]*15+[1]+[0]*4

test = np.array([standard, gpa, grad, inter, abr, cert, club])
print(model.predict_proba(test))

[[0.76186664 0.23813336]
 [0.83768089 0.16231911]
 [0.80450708 0.19549292]
 [0.79217602 0.20782398]
 [0.71186687 0.28813313]
 [0.70781202 0.29218798]
 [0.81604637 0.18395363]]


In [14]:
a = model.predict_proba(test)

In [15]:
dataset = pd.DataFrame({'Column1': a[:, 0], 'Column2': a[:, 1]})

In [16]:
dataset

,Column1,Column2
0,0.761867,0.238133
1,0.837681,0.162319
2,0.804507,0.195493
3,0.792176,0.207824
4,0.711867,0.288133
5,0.707812,0.292188
6,0.816046,0.183954


In [17]:
standard = [1,1,1,1,1,1]+[1]+[0]*10+[1]+[0]*15+[1]+[0]*4
bs = [1,1,1,1,1,1]+[1]+[0]*10+[0]*1+[1]+[0]*14+[1]+[0]*4
dg = [1,1,1,1,1,1]+[1]+[0]*10+[0]*2+[1]+[0]*13+[1]+[0]*4
ic = [1,1,1,1,1,1]+[1]+[0]*10+[0]*3+[1]+[0]*12+[1]+[0]*4
gj = [1,1,1,1,1,1]+[1]+[0]*10+[0]*4+[1]+[0]*11+[1]+[0]*4
dj = [1,1,1,1,1,1]+[1]+[0]*10+[0]*5+[1]+[0]*10+[1]+[0]*4
ws = [1,1,1,1,1,1]+[1]+[0]*10+[0]*6+[1]+[0]*9+[1]+[0]*4
gg = [1,1,1,1,1,1]+[1]+[0]*10+[0]*7+[1]+[0]*8+[1]+[0]*4
gw = [1,1,1,1,1,1]+[1]+[0]*10+[0]*8+[1]+[0]*7+[1]+[0]*4
cb = [1,1,1,1,1,1]+[1]+[0]*10+[0]*9+[1]+[0]*6+[1]+[0]*4
cn = [1,1,1,1,1,1]+[1]+[0]*10+[0]*10+[1]+[0]*5+[1]+[0]*4
jb = [1,1,1,1,1,1]+[1]+[0]*10+[0]*11+[1]+[0]*4+[1]+[0]*4
jn = [1,1,1,1,1,1]+[1]+[0]*10+[0]*12+[1]+[0]*3+[1]+[0]*4
gb = [1,1,1,1,1,1]+[1]+[0]*10+[0]*13+[1]+[0]*2+[1]+[0]*4
gn = [1,1,1,1,1,1]+[1]+[0]*10+[0]*14+[1]+[0]*1+[1]+[0]*4
jj = [1,1,1,1,1,1]+[1]+[0]*10+[0]*15+[1]+[1]+[0]*4


test_location = np.array([standard, bs, dg, ic, gj, dj, ws, gg, gw, cb, cn, jb, jn, gb, gn , jj])
a = model.predict_proba(test_location)
data2 = pd.DataFrame({'Column1': a[:, 0], 'Column2': a[:, 1]})
data2

,Column1,Column2
0,0.761867,0.238133
1,0.713595,0.286405
2,0.821207,0.178793
3,0.792476,0.207524
4,0.782643,0.217357
5,0.830477,0.169523
6,0.818940,0.181060
7,0.790916,0.209084
8,0.766503,0.233497
9,0.804167,0.195833


In [18]:
standard = [1,1,1,1,1,1]+[1]+[0]*10+[1]+[0]*15+[1]+[0]*4
two = [1,1,1,1,1,1]+[0]+[1]+[0]*9+[1]+[0]*15+[1]+[0]*4
ind = [1,1,1,1,1,1]+[0]*2+[1]+[0]*8+[1]+[0]*15+[1]+[0]*4
edu = [1,1,1,1,1,1]+[0]*3+[1]+[0]*7+[1]+[0]*15+[1]+[0]*4
tel = [1,1,1,1,1,1]+[0]*4+[1]+[0]*6+[1]+[0]*15+[1]+[0]*4
cyber = [1,1,1,1,1,1]+[0]*5+[1]+[0]*5+[1]+[0]*15+[1]+[0]*4
poly = [1,1,1,1,1,1]+[0]*6+[1]+[0]*4+[1]+[0]*15+[1]+[0]*4
none = [1,1,1,1,1,1]+[0]*7+[1]+[0]*3+[1]+[0]*15+[1]+[0]*4
cyber1 = [1,1,1,1,1,1]+[0]*8+[1]+[0]*2+[1]+[0]*15+[1]+[0]*4
poly1 = [1,1,1,1,1,1]+[0]*9+[1]+[0]*1+[1]+[0]*15+[1]+[0]*4
poly3 = [1,1,1,1,1,1]+[0]*10+[1]+[1]+[0]*15+[0]*3+[1]+[0]

test_unimajor = np.array([standard, two, ind, edu, tel, cyber, poly, none,cyber1,poly1,poly3])
a = model.predict_proba(test_unimajor)
data3 = pd.DataFrame({'Column1': a[:, 0], 'Column2': a[:, 1]})
data3

,Column1,Column2
0,0.761867,0.238133
1,0.841668,0.158332
2,0.812651,0.187349
3,0.849087,0.150913
4,0.855996,0.144004
5,0.763028,0.236972
6,0.762981,0.237019
7,0.806150,0.193850
8,0.810124,0.189876
9,0.828533,0.171467


In [19]:
standard = [1,1,1,1,1,1]+[1]+[0]*10+[1]+[0]*15+[1]+[0]*4
two = [1,1,1,1,1,1]+[1]+[0]*10+[1]+[0]*15+[0]+[1]+[0]*3
ind = [1,1,1,1,1,1]+[1]+[0]*10+[1]+[0]*15+[0]*2+[1]+[0]*2
edu = [1,1,1,1,1,1]+[1]+[0]*10+[1]+[0]*15+[0]*3+[1]+[0]
tel = [1,1,1,1,1,1]+[1]+[0]*10+[1]+[0]*15+[0]*4+[1]

test_unimajor = np.array([standard, two, ind, edu, tel])
a = model.predict_proba(test_unimajor)
data4 = pd.DataFrame({'Column1': a[:, 0], 'Column2': a[:, 1]})
data4

,Column1,Column2
0,0.761867,0.238133
1,0.690359,0.309641
2,0.692917,0.307083
3,0.720860,0.279140
4,0.753007,0.246993


In [20]:
score_perm = pd.concat([dataset,data3,data2,data4],axis=0)

In [21]:
len(score_perm)

39

In [22]:
col = X.columns.tolist()

In [23]:
col = ['standard'] + col

In [24]:
data = pd.DataFrame({'variable': col})

In [25]:
score_perm = score_perm.reset_index()

In [26]:
data

,variable
0,standard
1,gpaIdc
2,certIdc
3,abrIdc
4,internIdc
5,gradIdc
6,clubIdc
7,uniMajor_1
8,uniMajor_2
9,uniMajor_3


In [27]:
score_perm

,index,Column1,Column2
0,0,0.761867,0.238133
1,1,0.837681,0.162319
2,2,0.804507,0.195493
3,3,0.792176,0.207824
4,4,0.711867,0.288133
5,5,0.707812,0.292188
6,6,0.816046,0.183954
7,0,0.761867,0.238133
8,1,0.841668,0.158332
9,2,0.812651,0.187349


In [28]:
score_perm = pd.concat([data,score_perm],axis=1).drop('index',axis=1)

In [29]:
score_perm.to_csv('score_perm.csv')

# Logistic Regression - 대기업/중소기업

In [38]:
X=X2
y = df2['corpSize']

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_validTest, y_train, y_validTest = train_test_split(X, y, train_size=0.6, test_size = 0.4, random_state=1234) 
X_valid, X_test, y_valid, y_test = train_test_split(X_validTest, y_validTest, train_size = 0.5, test_size = 0.5, random_state = 1234)

In [40]:
from sklearn.linear_model import LogisticRegression
m1 = LogisticRegression(max_iter = 4000)
m1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=4000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [41]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=m1, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_valid, y_valid)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.754411 using {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.754411 (0.048598) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.754411 (0.048598) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.754411 (0.048598) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.753653 (0.049884) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.753653 (0.049884) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.753653 (0.049884) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.754394 (0.045809) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.754394 (0.045809) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.754394 (0.045809) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.749865 (0.034877) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.749865 (0.034877) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.751397 (0.041856) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.746061 (0.007994) wit

In [42]:
model = LogisticRegression( C= 100, penalty= 'l2',solver= 'newton-cg')
model.fit(X_train, y_train)

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
X.columns

Index(['sex', 'gpaIdc', 'engIdc', 'engconvIdc', 'contIdc', 'volIdc',
       'uniType_1', 'uniType_2', 'uniType_3', 'uniType_4', 'uniType_7',
       'uniType_19990920', 'uniMajor_1', 'uniMajor_2', 'uniMajor_3',
       'uniMajor_4', 'uniMajor_5', 'uniMajor_6', 'uniMajor_7', 'uniMajor_8',
       'uniMajor_10', 'uniMajor_19990920', 'youthHouse_1', 'youthHouse_2',
       'youthHouse_3', 'youthHouse_4'],
      dtype='object')

In [63]:
standard = [0,1,1,1,1,1]+[1]+[0]*5+[1]+[0]*9+[1]+[0]*3
gpa = [1,1,1,1,1,1]+[1]+[0]*5+[1]+[0]*9+[1]+[0]*3
grad = [0,0,1,1,1,1]+[1]+[0]*5+[1]+[0]*9+[1]+[0]*3
inter = [0,1,0,1,1,1]+[1]+[0]*5+[1]+[0]*9+[1]+[0]*3
abr = [0,1,1,0,1,1]+[1]+[0]*5+[1]+[0]*9+[1]+[0]*3
cert = [0,1,1,1,0,1]+[1]+[0]*5+[1]+[0]*9+[1]+[0]*3
club = [0,1,1,1,1,0]+[1]+[0]*5+[1]+[0]*9+[1]+[0]*3

test = np.array([standard, gpa, grad, inter, abr, cert, club])
a = model.predict_proba(test)
data1 = pd.DataFrame({'Column1': a[:, 0], 'Column2': a[:, 1]})
data1

,Column1,Column2
0,0.715577,0.284423
1,0.796982,0.203018
2,0.650828,0.349172
3,0.637118,0.362882
4,0.675891,0.324109
5,0.764943,0.235057
6,0.654682,0.345318


In [64]:
standard = [0,1,1,1,1,1]+[1]+[0]*5+[1]+[0]*9+[1]+[0]*3
two = [0,1,1,1,1,1]+[0]+[1]+[0]*4+[1]+[0]*9+[1]+[0]*3
ind = [0,1,1,1,1,1]+[0]*2+[1]+[0]*3+[1]+[0]*9+[1]+[0]*3
edu = [0,1,1,1,1,1]+[0]*3+[1]+[0]*2+[1]+[0]*9+[1]+[0]*3
tel = [0,1,1,1,1,1]+[0]*4+[1]+[0]*1+[1]+[0]*9+[1]+[0]*3
cyber = [0,1,1,1,1,1]+[0]*5+[1]+[0]*9+[1]+[1]+[0]*3

test_unimajor = np.array([standard, two, ind, edu, tel, cyber])
a = model.predict_proba(test_unimajor)
data2 = pd.DataFrame({'Column1': a[:, 0], 'Column2': a[:, 1]})
data2

,Column1,Column2
0,0.715577,0.284423
1,0.834548,0.165452
2,0.875312,0.124688
3,0.212818,0.787182
4,0.871011,0.128989
5,0.756973,0.243027


In [65]:
standard = [0,1,1,1,1,1]+[1]+[0]*5+[1]+[0]*9+[1]+[0]*3
two = [0,1,1,1,1,1]+[1]+[0]*5+[0]+[1]+[0]*8+[1]+[0]*3
ind = [0,1,1,1,1,1]+[1]+[0]*5+[0]*2+[1]+[0]*7+[1]+[0]*3
edu = [0,1,1,1,1,1]+[1]+[0]*5+[0]*3+[1]+[0]*6+[1]+[0]*3
tel = [0,1,1,1,1,1]+[1]+[0]*5+[0]*4+[1]+[0]*5+[1]+[0]*3
cyber = [0,1,1,1,1,1]+[1]+[0]*5+[0]*5+[1]+[0]*4+[1]+[0]*3
a1 = [0,1,1,1,1,1]+[1]+[0]*5+[0]*6+[1]+[0]*3+[1]+[0]*3
a2 = [0,1,1,1,1,1]+[1]+[0]*5+[0]*7+[1]+[0]*2+[1]+[0]*3
a3 = [0,1,1,1,1,1]+[1]+[0]*5+[0]*8+[1]+[0]+[1]+[0]*3
a4 = [0,1,1,1,1,1]+[0]*5+[1]+[0]*9+[1]+[1]+[0]*3



test_unimajor = np.array([standard, two, ind, edu, tel, cyber, a1,a2,a3,a4])
a = model.predict_proba(test_unimajor)
data3 = pd.DataFrame({'Column1': a[:, 0], 'Column2': a[:, 1]})
data3

,Column1,Column2
0,0.715577,0.284423
1,0.671813,0.328187
2,0.831889,0.168111
3,0.751282,0.248718
4,0.774388,0.225612
5,0.717568,0.282432
6,0.820243,0.179757
7,0.259799,0.740201
8,0.824556,0.175444
9,0.756973,0.243027


In [66]:
standard = [0,1,1,1,1,1]+[1]+[0]*5+[1]+[0]*9+[1]+[0]*3
two = [0,1,1,1,1,1]+[1]+[0]*5+[1]+[0]*9+[0]+[1]+[0]*2
a2 = [0,1,1,1,1,1]+[1]+[0]*5+[1]+[0]*9+[0]*2+[1]+[0]
a1 = [0,1,1,1,1,1]+[1]+[0]*5+[1]+[0]*9+[0]*3+[1]

test_unimajor = np.array([standard, two, a2, a1])
a = model.predict_proba(test_unimajor)
data4 = pd.DataFrame({'Column1': a[:, 0], 'Column2': a[:, 1]})
data4

,Column1,Column2
0,0.715577,0.284423
1,0.719316,0.280684
2,0.823116,0.176884
3,0.245579,0.754421


In [62]:
df2[df2['uniMajor']==19990920]

,sex,gpaIdc,engIdc,engconvIdc,contIdc,volIdc,uniType,uniMajor,youthHouse,corpSize
0,1,2,2,2,2,2,19990920,19990920,1,0
11,2,2,2,2,2,2,19990920,19990920,1,0
23,1,2,2,1,2,2,19990920,19990920,2,1
25,1,2,2,2,2,2,19990920,19990920,1,0
35,2,2,2,2,2,2,19990920,19990920,1,0
...,...,...,...,...,...,...,...,...,...,...
2117,1,2,2,2,2,2,19990920,19990920,1,0
2131,1,1,1,2,2,2,19990920,19990920,1,0
2169,1,2,2,2,2,2,19990920,19990920,1,0
2170,1,2,2,2,2,2,19990920,19990920,2,0


In [80]:
score_corp = pd.concat([data1,data2,data3,data4],axis=0).reset_index()
col = ['standard'] + X.columns.tolist()
data = pd.DataFrame({'variable': col})
score_corp = pd.concat([data,score_corp],axis=1).drop('index',axis=1)

In [81]:
score_corp

,variable,Column1,Column2
0,standard,0.715577,0.284423
1,sex,0.796982,0.203018
2,gpaIdc,0.650828,0.349172
3,engIdc,0.637118,0.362882
4,engconvIdc,0.675891,0.324109
5,contIdc,0.764943,0.235057
6,volIdc,0.654682,0.345318
7,uniType_1,0.715577,0.284423
8,uniType_2,0.834548,0.165452
9,uniType_3,0.875312,0.124688


In [82]:
score_corp.to_csv('score_corp.csv')